### Riddler Classic: 


#### Handling the map:
Going to treat the roads like a clock: 

- 12 o'clock = 1

- 3 o'clock = 3

- 6 o'clock = 5

- 9 o'clock = 7

#### process:

- Exploration of this is here: https://github.com/dwanneruchi/538_Riddlers/blob/master/20210618/riddler_classic_process.ipynb

- General rules:
    - A crash is missed only in the following scenarios: 
        1) Car A is inside of Car B route (e.g. Car A: 2 -> 3, Car B: 1 -> 6)
        2) Car B is inside of Car A route (e.g. Car A: 6 -> 2, Car B: 3 -> 5)
        3) Car A & Car B are totally disjoint (e.g. Car A: 2 -> 7, Car B: 8 -> 1)
    - Anything else would qualify as a crash:
        - Assumption 1: If Car A Leaves 1 and Lands in 2, and Car B Leaves 4 and Lands in 1 that this qualifies as a crash.
            - `@xaqwg` on Twitter said as much: https://twitter.com/Donald_Adamek/status/1405879172019404800
        - Assumption 2: If Car A & B end in the same place, also counts as a Crash (confirmed in riddler) 

In [1]:
import random 
import time 

class Car:
    def __init__(self, point_list):
        self.origin_a = random.choices(point_list, k=1)[0]
        self.origin_b = random.choices([x for x in point_list if x != self.origin_a], k=1)[0]
        self.endpoint_a = random.choices([x for x in point_list if x != self.origin_a], k=1)[0]
        self.endpoint_b = random.choices([x for x in point_list if x != self.origin_b], k=1)[0]
        self.car_a = (self.origin_a, self.endpoint_a)
        self.car_b = (self.origin_b, self.endpoint_b)
    
    def determine_crossing(self):
        """ Function returns a 1 for a crash, 0 for no crash
        
        Info: 
        1) i always treat start as lowest value, end as highest. this lets me avoid
           some messier logic. It handles instances where drivers end on the same path but started form
           different ends (e.g. (1,4), (8,4)) since a crash is considered if same road is EVER traveled
           by Driver A & B. 
        
        2) The crashes handled are: path crossing, starting / ending same route
        
        3) The only way to not crash is for:
           - One car to be completely enclosed by the route of another (e.g. Car A (1,8), Car B (2,6))
           - One car to be completely outside of the range of another (e.g. Car A (1,2), Car B (4,6))
        
        """
        # capture start, end
        o_a, e_a = min(self.car_a), max(self.car_a)
        o_b, e_b = min(self.car_b), max(self.car_b)

        if o_a < o_b < e_a and o_a < e_b < e_a:
            return 0

        # route is totally outside 
        if (not(o_a <= o_b <= e_a)) and (not(o_a <= e_b <= e_a)):
            return 0

        # assume no crash
        return 1

### Quick Test

The testing below is to ensure proper results, so I override the `self.car_a`, `self.car_b` attributes

In [2]:
# test instance 
test_car = Car(list(range(1,8)))

# second car is between roads 
test_car.car_a = (1,8)
test_car.car_b = (2,3)
assert(test_car.determine_crossing() == 0)

# first car is between roads
test_car.car_a = (3,6)
test_car.car_b = (1,8)
assert(test_car.determine_crossing() == 0)

# second car would cross here since going to same location, so crashes
test_car.car_a = (1,3)
test_car.car_b = (4,3)
assert(test_car.determine_crossing() == 1)

# another crash 
test_car.car_a = (1,5)
test_car.car_b = (8,2)
assert(test_car.determine_crossing() == 1)

# no crash as they are totally separate
test_car.car_a = (3,8)
test_car.car_b = (7,4)
assert(test_car.determine_crossing() == 0)

### Simulation: 

- Lists are slow so switching to `NumPy` later makes sense
- A `class` was totally unecessary here....passing random values through a matrix would be way faster.
    - Initially I was thinking I would need to reference one class within another, but took a different approach. 

In [3]:
sim_list = [100, 1_000, 10_000, 100_000, 
            1_000_000, 2_000_000, 5_000_000]

for sim_count in sim_list:
    start_time = time.time()
    crashes = 0
    for _ in range(sim_count):
        my_car = Car(list(range(1,8)))
        crashes += my_car.determine_crossing()

    end_time = time.time()
    print(f"Time for {sim_count} sims: {end_time - start_time:.2f}")
    print(f"Likelihood of crash: {crashes / sim_count:.3f}")

Time for 100 sims: 0.01
Likelihood of crash: 0.660
Time for 1000 sims: 0.06
Likelihood of crash: 0.613
Time for 10000 sims: 0.13
Likelihood of crash: 0.636
Time for 100000 sims: 1.08
Likelihood of crash: 0.627
Time for 1000000 sims: 11.97
Likelihood of crash: 0.630
Time for 2000000 sims: 27.71
Likelihood of crash: 0.630
Time for 5000000 sims: 51.46
Likelihood of crash: 0.629


### Extra Credit: What do we Converge to As Routes -> infinity

- Going to just use 10_000 sims, over an ever-expanding range up to 1000 (not the best, but gives an approximation)
- Again, using `NumPy` would make WAY more sense.....and would be WAYYYY faster and would allow for a MUCHHH higher max. Need to rewrite. 

In [13]:
route_size = 10
sim_count = 10_000
while route_size < 1_000:
    crashes = 0
    for _ in range(sim_count):
        my_car = Car(list(range(1,route_size)))
        crashes += my_car.determine_crossing()
    # print every 1_000_000
    if route_size % 100 == 0:
        print(f"Likelihood of crash with {route_size} routes: {crashes / sim_count:.3f}")
    route_size += 10

Likelihood of crash with 100 routes: 0.361
Likelihood of crash with 200 routes: 0.350
Likelihood of crash with 300 routes: 0.337
Likelihood of crash with 400 routes: 0.338
Likelihood of crash with 500 routes: 0.348
Likelihood of crash with 600 routes: 0.331
Likelihood of crash with 700 routes: 0.335
Likelihood of crash with 800 routes: 0.335
Likelihood of crash with 900 routes: 0.334
